### NORMALIZE CREATING NEW TABLE

ta metoda polega na tworzeniu tabeli dla każdej listy

In [27]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

In [28]:
paths_test = [ '../data-test/airlines.json', 
            '../data-test/gists.json', 
            '../data-test/historical-events.json',#have to rename the same value in json (movie on  the same level)
            '../data-test/movies.json',
            '../data-test/reddit.json',#bad
            '../data-test/nasa.json'
            ]

In [2]:
paths = [ '../data/airlines.json', 
            '../data/gists.json', 
            '../data/historical-events.json',
            '../data/movies.json',
            '../data/reddit.json',
            '../data/nasa.json']

In [1]:
def make_new_json(lists,table_name):
    data = []
    for i in range(len(lists)):
        if lists[i] is not None and not isinstance(lists[i], float):
            for item in lists[i]:
                if isinstance(item, dict):
                    item_data = item.copy()
                    item_data["row_number"] = i
                else:
                    item_data={
                        "row_number": i,
                        "value": item
                    }
                data.append(item_data)
    return data


def delete_lists(df,name):

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) and i for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()
        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    tables = {name: main_table}

    for dropped_list in list_columns:
        new_json=make_new_json(list_df,dropped_list)
        tables[dropped_list]=delete_lists(new_json,dropped_list)

    return tables

results=[]
for path in paths:
    with open(path, encoding='utf-8') as f:
        dt = json.load(f)

    tables = delete_lists(dt,'main')
    results.append(tables)

#change tables from list to dict with names


NameError: name 'paths' is not defined

In [ ]:
def make_new_json(lists,table_name):
    data = []
    for i in range(len(lists)):
        if lists[i] is not None and not isinstance(lists[i], float):
            for item in lists[i]:
                if isinstance(item, dict):
                    item_data = item.copy()
                    item_data["row_number"] = i
                else:
                    item_data={
                        "row_number": i,
                        "value": item
                    }
                data.append(item_data)
    return data

def delete_lists(df, name):
    df = pd.json_normalize(df)
    is_list = df.apply(lambda x: any(isinstance(i, list) and i for i in x))
    list_columns = is_list[is_list].index.tolist()

    if list_columns:
        list_df = df[list_columns]
        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)

    main_table = pa.Table.from_pandas(df)
    tables = {name: main_table}

    for dropped_list in list_columns:
        new_json = make_new_json(list_df, dropped_list)
        tables[dropped_list] = delete_lists(new_json, dropped_list)

    return tables

results = [delete_lists(json.load(open(path, encoding='utf-8')), 'main') for path in paths]


In [46]:
results[4]

[pyarrow.Table
 kind: string
 data.after: string
 data.dist: int64
 data.modhash: string
 data.geo_filter: null
 data.before: null
 row_number: int64
 ----
 kind: [["Listing"]]
 data.after: [["t3_16q4vh8"]]
 data.dist: [[25]]
 data.modhash: [[""]]
 data.geo_filter: [1 nulls]
 data.before: [1 nulls]
 row_number: [[1]],
 pyarrow.Table
 kind: string
 row_number: int64
 data.approved_at_utc: null
 data.subreddit: string
 data.selftext: string
 data.author_fullname: string
 data.saved: bool
 data.mod_reason_title: null
 data.gilded: int64
 data.clicked: bool
 data.title: string
 data.subreddit_name_prefixed: string
 data.hidden: bool
 data.pwls: double
 data.link_flair_css_class: string
 data.downs: int64
 data.thumbnail_height: int64
 data.top_awarded_type: null
 data.hide_score: bool
 data.name: string
 data.quarantine: bool
 data.link_flair_text_color: string
 data.upvote_ratio: double
 data.author_flair_background_color: string
 data.subreddit_type: string
 data.ups: int64
 data.total_a